In [95]:
from base64 import b64encode

In [75]:
import jinja2
import json
def jinga2TemplateString(TEMPLATE_STRING, args={}):
    return jinja2.Environment(loader=jinja2.BaseLoader).from_string(TEMPLATE_STRING).render(**args)

In [76]:
self_execution_template  = """(function(){
{{ content }}
})()"""

# Lookup Array
This is a method i was working on but its not the best

In [77]:
code_template  ="""
let e = Array.from(Array(95)).map((e, i) => i + 32).map((x) => String.fromCharCode(x));
let d = {{code}};
let o = '';
for (let i = 0; i < d.length; i++){
o+= e[d[i]];
};
console.log(o);
window[{% for c in exec_cmd %}e[{{c}}]+{% endfor %}''](o);
""".strip()

In [117]:
start = 32
end = 32 + 95
e = [chr(c) for c in range(start, end)]

In [118]:
command = 'eval'
exec_cmd = [e.index(c) for c in list(command)]

In [119]:
code = """
alert("This is powerful.")
"""

In [120]:
def encode_script(code):
    out = []
    for letter in list(code.strip()):
        out.append(e.index(letter))
    return out

In [121]:
out = []
for letter in list(code.strip()):
    out.append(e.index(letter))
json.dumps(out)

In [122]:
out = jinga2TemplateString(code_template,{'code':json.dumps(out),'exec_cmd':exec_cmd})
print(out)

In [123]:
out = jinga2TemplateString(self_execution_template,{'content':out})
print(out)

In [124]:
out = out.replace("\t",' ').replace("\n",' ').replace('  ',' ').replace(', ',',').replace(' = ','=')
out

In [137]:
code = """
var a = "Hello ";
var b = "World!";
var c =a+b;
alert(c);"""

code = code.strip().encode('ascii')
string = b64encode(code).decode('ascii')
print(string)
print()
string = list(string)
print(string)
print()
for index, letter in enumerate(string):
    string[index] = hex(ord(letter))
print(string)
print()

string = ["\\x"+s.lstrip('0x') for s in string]
print(string)
print()

out = ''.join(string)
print(out)

# Function Injection from encoded string
working: Chrome


In [138]:
print(jinga2TemplateString("""Function(atob("{{code}}"))()""",{'code':out}))

# Append Blob url as script to body
Working: Chrome


In [139]:
print(jinga2TemplateString("""
var script = document.createElement('script');
script.type='text/javascript';
script.src = URL.createObjectURL(new Blob([atob("{{code}}")]));
document.body.appendChild(script);
""",{'code':out}))

# Service Worker Blob 
Not Working in: Google Chrome

In [140]:
print(jinga2TemplateString("""
var url= URL.createObjectURL(new Blob([atob("{{code}}")]));
navigator.serviceWorker.register(url, {
  updateUrl: '/sw.js'
})
""",{'code':out}))

# Timeout Based
Working

In [141]:
print(jinga2TemplateString("""
setTimeout(new Function(atob("{{code}}")),0)
""",{'code':out}))

In [142]:
print(jinga2TemplateString("""
setTimeout(eval(atob("{{code}}")),0)
""",{'code':out}))

In [143]:
print(jinga2TemplateString("""
eval(atob("{{code}}"));
""",{'code':out}))

In [206]:
filename="SomeCode.js"
print(jinga2TemplateString("""
var url= URL.createObjectURL(new Blob([atob("{{code}}")],{type:'octet/stream'}));
var myAnchor = document.createElement('a');
myAnchor.download = '{{filename}}';
myAnchor.href = url;
myAnchor.click();
""",{'code':out,'filename':filename}))



In [176]:
out

In [198]:
import random


In [208]:
used = []
script = []
for index,start in enumerate(range(0,len(out),32)):
    letter = random.choice(['p','y','r','b','v','a','z','e'])+str(random.randint(0,99999)).ljust(5,'0')
    used.append(letter)
    script.append(jinga2TemplateString("""var {{letter}} = "{{code}}";""",{'letter':letter,'code':out[start:start+32]}))
script.append(jinga2TemplateString("""var code = atob({% for letter in letters %}{{letter}}+{% endfor %}"");""",{'letters':used}))
print("\n".join(script))

In [181]:
print(jinga2TemplateString("""
[]["filter"]["constructor"](atob("{{code}}"))()
""",{'code':out}))

# filter Constructor
working: chrome

In [147]:
print(jinga2TemplateString("""
[]["filter"]["constructor"](atob("{{code}}"))()
""",{'code':out}))